# Optimisation of a Neocortical Layer 5 Pyramidal Cell

This notebook shows you how to optimise the maximal conductance of Neocortical Layer 5 Pyramidal Cell as used in Markram et al. 2015.

Author of this script: Werner Van Geit @ Blue Brain Project

Choice of parameters, protocols and other settings was done by Etay Hay @ HUJI

What's described here is a more advanced use of BluePyOpt. We suggest to first go through the introductary example here: https://github.com/BlueBrain/BluePyOpt/blob/master/examples/simplecell/simplecell.ipynb

**If you use the methods in this notebook, we ask you to cite the following publications when publishing your research:**

Van Geit, W., M. Gevaert, G. Chindemi, C. Rössert, J.-D. Courcol, E. Muller, F. Schürmann, I. Segev, and H. Markram (2016, March). BluePyOpt: Leveraging open source software and cloud infrastructure to optimise model parameters in neuroscience. ArXiv e-prints.
http://arxiv.org/abs/1603.00500

Markram, H., E. Muller, S. Ramaswamy, M. W. Reimann, M. Abdellah, C. A. Sanchez, A. Ailamaki, L. Alonso-Nanclares, N. Antille, S. Arsever, et al. (2015). Reconstruction and simulation of neocortical microcircuitry. Cell 163(2), 456–492.
http://www.cell.com/abstract/S0092-8674%2815%2901191-5

Some of the modules loaded in this script are located in the L5PC example folder: https://github.com/BlueBrain/BluePyOpt/tree/master/examples/l5pc 

We first load the bluepyopt python module, the ephys submodule and some helper functionality

In [1]:
%load_ext autoreload
%autoreload

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import pprint
pp = pprint.PrettyPrinter(indent=2)

%matplotlib notebook
import matplotlib.pyplot as plt

Enable the code below to enable debug level logging

In [2]:
# import logging                                                                      
# logging.basicConfig()                                                               
# logger = logging.getLogger()                                                        
# logger.setLevel(logging.DEBUG)   

## Model description

### Morphology

We're using a complex reconstructed morphology of an L5PC cell. Let's visualise this with the BlueBrain NeuroM software:

In [3]:
!pip install neurom --upgrade
import neurom.ezy
import neurom.view
neurom.view.view.neuron(neurom.ezy.load_neuron('morphology/C060114A7.asc'));

Downloading/unpacking neurom
Cleaning up...
setuptools must be installed to install from a source distribution
Storing debug log for failure in /Users/werner/.pip/pip.log


<IPython.core.display.Javascript object>

To load the morphology we create a NrnFileMorphology object. We set 'do_replace_axon' to True to replace the axon with a AIS.

In [4]:
morphology = ephys.morphologies.NrnFileMorphology('morphology/C060114A7.asc', do_replace_axon=True)
print str(morphology)

morphology/C060114A7.asc


### Parameters

Since we have many parameters in this model, they are stored in a json file: https://github.com/BlueBrain/BluePyOpt/blob/master/examples/l5pc/config/parameters.json

In [5]:
import json
param_configs = json.load(open('config/parameters.json'))
print [param_config['param_name'] for param_config in param_configs]

[u'g_pas', u'e_pas', u'cm', u'Ra', u'v_init', u'celsius', u'ena', u'ek', u'cm', u'ena', u'ek', u'cm', u'ena', u'ek', u'gIhbar_Ih', u'gNaTs2_tbar_NaTs2_t', u'gSKv3_1bar_SKv3_1', u'gImbar_Im', u'gIhbar_Ih', u'gNaTa_tbar_NaTa_t', u'gNap_Et2bar_Nap_Et2', u'gK_Pstbar_K_Pst', u'gK_Tstbar_K_Tst', u'gSK_E2bar_SK_E2', u'gSKv3_1bar_SKv3_1', u'gCa_HVAbar_Ca_HVA', u'gCa_LVAstbar_Ca_LVAst', u'gamma_CaDynamics_E2', u'decay_CaDynamics_E2', u'gNaTs2_tbar_NaTs2_t', u'gSKv3_1bar_SKv3_1', u'gSK_E2bar_SK_E2', u'gCa_HVAbar_Ca_HVA', u'gCa_LVAstbar_Ca_LVAst', u'gamma_CaDynamics_E2', u'decay_CaDynamics_E2', u'gIhbar_Ih']


The directory that contains this notebook has a module that will load all the parameters in BluePyOpt Parameter objects

In [6]:
import l5pc_model
parameters = l5pc_model.define_parameters()
print '\n'.join('%s' % param for param in parameters)

g_pas.all: ['all'] g_pas = 3e-05
e_pas.all: ['all'] e_pas = -75
cm.all: ['all'] cm = 1
Ra.all: ['all'] Ra = 100
v_init: v_init = -65
celsius: celsius = 34
ena.apical: ['apical'] ena = 50
ek.apical: ['apical'] ek = -85
cm.apical: ['apical'] cm = 2
ena.somatic: ['somatic'] ena = 50
ek.somatic: ['somatic'] ek = -85
cm.basal: ['basal'] cm = 2
ena.axonal: ['axonal'] ena = 50
ek.axonal: ['axonal'] ek = -85
gIhbar_Ih.basal: ['basal'] gIhbar_Ih = 8e-05
gNaTs2_tbar_NaTs2_t.apical: ['apical'] gNaTs2_tbar_NaTs2_t = [0, 0.04]
gSKv3_1bar_SKv3_1.apical: ['apical'] gSKv3_1bar_SKv3_1 = [0, 0.04]
gImbar_Im.apical: ['apical'] gImbar_Im = [0, 0.001]
gIhbar_Ih.apical: ['apical'] gIhbar_Ih = 8e-05
gNaTa_tbar_NaTa_t.axonal: ['axonal'] gNaTa_tbar_NaTa_t = [0, 4]
gNap_Et2bar_Nap_Et2.axonal: ['axonal'] gNap_Et2bar_Nap_Et2 = [0, 4]
gK_Pstbar_K_Pst.axonal: ['axonal'] gK_Pstbar_K_Pst = [0, 1]
gK_Tstbar_K_Tst.axonal: ['axonal'] gK_Tstbar_K_Tst = [0, 0.1]
gSK_E2bar_SK_E2.axonal: ['axonal'] gSK_E2bar_SK_E2 = [0, 0.1

As you can see there are two types of parameters, parameters with a fixed value and parameters with bounds. The latter will be optimised by the algorithm.

### Mechanism

We also need to add all the necessary mechanisms, like ion channels to the model. 
The configuration of the mechanisms is also stored in a json file, and can be loaded in a similar way.

In [7]:
mechanisms = l5pc_model.define_mechanisms()
print '\n'.join('%s' % mech for mech in mechanisms)

IOError: [Errno 2] No such file or directory: 'mechanisms.json'

# Cell model

With the morphology, mechanisms and parameters we can build the cell model

In [ ]:
l5pc_cell = ephys.models.CellModel('l5pc', morph=morphology, mechs=mechanisms, params=parameters)
print l5pc_cell

For use in the cell evaluator later, we need to make a list of the name of the parameters we are going to optimise.
These are the parameters that are not frozen.

In [ ]:
param_names = [param.name                                                       
                    for param in l5pc_cell.params.values()                           
                   if not param.frozen]      

## Protocols

Now that we have a cell model, we can apply protocols to it. The protocols are also stored in a json file.

In [ ]:
proto_configs = json.load(open('config/protocols.json'))
print proto_configs

And they can be automatically loaded

In [ ]:
import l5pc_evaluator
fitness_protocols = l5pc_evaluator.define_protocols()
print '\n'.join('%s' % protocol for protocol in fitness_protocols.values())

## eFeatures

For every protocol we need to define which eFeatures will be used as objectives of the optimisation algorithm.

In [ ]:
feature_configs = json.load(open('config/features.json'))
pp.pprint(feature_configs)

In [ ]:
fitness_calculator = l5pc_evaluator.define_fitness_calculator(fitness_protocols)
print fitness_calculator

## Simulator

We need to define which simulator we will use. In this case it will be Neuron, i.e. the NrnSimulator class

In [ ]:
sim = ephys.simulators.NrnSimulator()

## Evaluator

With all the components defined above we can build a cell evaluator

In [ ]:
evaluator = ephys.evaluators.CellEvaluator(                                          
        cell_model=l5pc_cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)  

This evaluator can be used to run the protocols. The original parameter values for the Markram et al. 2015 L5PC model are:

In [ ]:
release_params = {
    'gNaTs2_tbar_NaTs2_t.apical': 0.026145,
    'gSKv3_1bar_SKv3_1.apical': 0.004226,
    'gImbar_Im.apical': 0.000143,
    'gNaTa_tbar_NaTa_t.axonal': 3.137968,
    'gK_Tstbar_K_Tst.axonal': 0.089259,
    'gamma_CaDynamics_E2.axonal': 0.002910,
    'gNap_Et2bar_Nap_Et2.axonal': 0.006827,
    'gSK_E2bar_SK_E2.axonal': 0.007104,
    'gCa_HVAbar_Ca_HVA.axonal': 0.000990,
    'gK_Pstbar_K_Pst.axonal': 0.973538,
    'gSKv3_1bar_SKv3_1.axonal': 1.021945,
    'decay_CaDynamics_E2.axonal': 287.198731,
    'gCa_LVAstbar_Ca_LVAst.axonal': 0.008752,
    'gamma_CaDynamics_E2.somatic': 0.000609,
    'gSKv3_1bar_SKv3_1.somatic': 0.303472,
    'gSK_E2bar_SK_E2.somatic': 0.008407,
    'gCa_HVAbar_Ca_HVA.somatic': 0.000994,
    'gNaTs2_tbar_NaTs2_t.somatic': 0.983955,
    'decay_CaDynamics_E2.somatic': 210.485284,
    'gCa_LVAstbar_Ca_LVAst.somatic': 0.000333
}


Running the responses is as easy as passing the protocols and parameters to the evaluator. (The line below will take some time to execute)

In [ ]:
release_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=release_params)

We can now plot all the responses

In [ ]:
def plot_responses(responses):
    fig, axes = plt.subplots(len(responses), figsize=(10,10))
    for index, (resp_name, response) in enumerate(sorted(responses.items())):
        axes[index].plot(response['time'], response['voltage'], label=resp_name)
        axes[index].set_title(resp_name)
    fig.tight_layout()
    fig.show()
plot_responses(release_responses)

Running an optimisation of the parameters now has become very easy. 
Of course running the L5PC optimisation will require quite some computing resources. 

To show a proof-of-concept, we will only run 2 generations, with 2 offspring individuals per generations.
If you want to run all full optimisation, you should run for 100 generations with an offspring size of 100 individuals. 

In [ ]:
opt = bpopt.optimisations.DEAPOptimisation(                                     
    evaluator=evaluator,                                                            
    offspring_size=2) 
final_pop, halloffame, log, hist = opt.run(max_ngen=2)

The first individual in the hall of fame will contain the best solution found.

In [ ]:
print halloffame[0]

These are the raw parameter values. 
The evaluator object can convert this in a dictionary, so that we can see the parameter names corresponding to these values.

In [ ]:
best_params = evaluator.param_dict(halloffame[0])
print pp.pprint(best_params)

Then we can run the fitness protocols on the model with these parameter values

In [ ]:
best_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=best_params)

And then we can also plot these responses. 

When you ran the above optimisation with only 2 individuals and 2 generations, this 'best' model will of course be very low quality.

In [ ]:
plot_responses(best_responses)